In [3]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 4.1 MB/s  0:00:04 eta 0:00:01


In [10]:
import os
import sys

# 1. 경로 설정 (반드시 모든 import보다 먼저 실행되어야 합니다!)
project_root = os.path.dirname(os.getcwd())
src_path = os.path.join(project_root, 'src')

if project_root not in sys.path:
    sys.path.insert(0, project_root) # 가장 우선순위로 추가
if src_path not in sys.path:
    sys.path.insert(0, src_path)     # src 폴더 내부도 직접 참조하도록 추가

print(f"✅ 프로젝트 루트 등록 완료: {project_root}")
print(f"✅ 소스 폴더 등록 완료: {src_path}")

# 2. 이제 모듈 임포트
try:
    # src 폴더가 등록되었으므로 이제 바로 불러올 수 있습니다.
    from k_league_timesfm_forecast import KLeagueForecaster
    print("🚀 딥러닝 예측 모델(KLeagueForecaster) 로드 성공!")
    HAS_NEW_TOOLS = True
except ImportError as e:
    HAS_NEW_TOOLS = False
    print(f"❌ 여전히 로드 실패: {e}")

# 3. BASE_DIR 설정 (데이터 로드용)
BASE_DIR = project_root

✅ 프로젝트 루트 등록 완료: /Users/sebokoh/데이터분석연습/데이콘/k리그데이터/리그데이터
✅ 소스 폴더 등록 완료: /Users/sebokoh/데이터분석연습/데이콘/k리그데이터/리그데이터/src
🚀 딥러닝 예측 모델(KLeagueForecaster) 로드 성공!


In [11]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import polars as pl
import warnings
warnings.filterwarnings("ignore")
import sys
import duckdb
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from IPython.display import display



#[New integration]
try:
    from k_league_timesfm_forecast import KLeagueForecaster
    from polars_duckdb_synergy import BigDataEngine
    HAS_NEW_TOOLS = True
except ImportError as e:
    HAS_NEW_TOOLS = False
    print(f" 신규 모델 로드 실패: {e}")


# [Advanced Toolkits] 선배들이 진짜 쓰고 싶어 하는 도구들
try:
    import sharp
    HAS_SHARP = True
except ImportError:
    HAS_SHARP = False

try:
    import timesfm
    HAS_TIMESFM = True
except ImportError:
    HAS_TIMESFM = False

try:
    import causalai
    HAS_CAUSAL = True
except ImportError:
    HAS_CAUSAL = False

try: from IPython.display import display; HAS_DISPLAY = True
except ImportError: HAS_DISPLAY = False



# 현재 노트북이 있는 폴더(notebooks)의 상위 폴더(프로젝트 루트)를 경로에 추가
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"현재 인식된 프로젝트 루트: {project_root}")


# 0. 환경 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="whitegrid")

# 현재 노트북 위치(notebooks/)에서 한 단계 위가 프로젝트 루트입니다.
BASE_DIR = os.path.dirname(os.getcwd())

# 경로가 잘 잡혔는지 확인용 출력
print(f"설정된 BASE_DIR: {BASE_DIR}")
RAW_DATA_PATH = os.path.join(BASE_DIR, "data/raw/raw_data.csv")
MATCH_INFO_PATH = os.path.join(BASE_DIR, "data/raw/match_info.csv")

def ensure_directories():
    for path in ["reports/figures", "reports/docs"]:
        os.makedirs(os.path.join(BASE_DIR, path), exist_ok=True)
    
def run_study_pipeline():
    ensure_directories()
    print(f" K리그 고도화 분석 파이프라인 시작: {datetime.datetime.now()}")
    print("="*80)

    


현재 인식된 프로젝트 루트: /Users/sebokoh/데이터분석연습/데이콘/k리그데이터/리그데이터
설정된 BASE_DIR: /Users/sebokoh/데이터분석연습/데이콘/k리그데이터/리그데이터


In [15]:
#----------------------------------------------------
# [Step 1] 데이터 로드
#----------------------------------------------------

print("\n[Step 1]DuckDB 데이터 로드")
con = duckdb.connect(database=':memory:')
raw_data = con.execute(f"SELECT * FROM read_csv_auto('{RAW_DATA_PATH}')").df()
match_info = con.execute(f"SELECT * FROM read_csv_auto('{MATCH_INFO_PATH}')").df()
print(f"완료: raw_data: {raw_data.shape}")




[Step 1]DuckDB 데이터 로드
완료: raw_data: (579306, 18)


In [ ]:
#------------------------------------
# [Step 2] 기본 집계
#------------------------------------
print("\n[STEP 2] 경기 단위 팀 통계 집계")
raw_data['result_name'] = raw_data['result_name'].fillna('Unknown')
